# Семантично ядро с интеграция на OpenBnB MCP сървър

Тази тетрадка демонстрира как да използвате Семантичното ядро с реалния OpenBnB MCP сървър, за да търсите истински Airbnb настанявания, използвайки MCPStdioPlugin. За достъп до LLM се използва Azure AI Foundry. За да настроите вашите променливи на средата, можете да следвате [Урок за настройка](/00-course-setup/README.md)


## Импортиране на необходимите пакети


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Създаване на връзка за MCP плъгин

Ще се свържем със [OpenBnB MCP сървъра](https://github.com/openbnb-org/mcp-server-airbnb), използвайки MCPStdioPlugin. Този сървър предоставя функционалност за търсене в Airbnb чрез пакета @openbnb/mcp-server-airbnb.


## Създаване на клиента

В този пример ще използваме Azure AI Foundry за достъп до LLM. Уверете се, че вашите променливи на средата са настроени правилно.


## Конфигурация на средата

Конфигурирайте настройките на Azure OpenAI. Уверете се, че сте задали следните променливи на средата:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Разбиране на интеграцията на OpenBnB MCP

Този ноутбук се свързва с **реалния OpenBnB MCP сървър**, който предоставя действителна функционалност за търсене в Airbnb.

### Как работи:

1. **MCPStdioPlugin**: Използва стандартна комуникация чрез вход/изход със сървъра MCP
2. **Реален NPM пакет**: Изтегля и стартира `@openbnb/mcp-server-airbnb` чрез npx
3. **Данни в реално време**: Връща реални данни за имоти от Airbnb чрез техните API
4. **Откриване на функции**: Агентът автоматично открива наличните функции от сървъра MCP

### Налични функции:

Сървърът OpenBnB MCP обикновено предоставя:
- **search_listings** - Търсене на имоти в Airbnb по местоположение и критерии
- **get_listing_details** - Получаване на подробна информация за конкретни имоти
- **check_availability** - Проверка на наличност за конкретни дати
- **get_reviews** - Извличане на отзиви за имоти
- **get_host_info** - Получаване на информация за домакините на имоти

### Предварителни изисквания:

- **Node.js** инсталиран на вашата система
- **Интернет връзка** за изтегляне на пакета на MCP сървъра
- **NPX** наличен (идва с Node.js)

### Тестване на връзката:

Можете да тествате сървъра MCP ръчно, като стартирате:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Това ще изтегли и стартира OpenBnB MCP сървъра, към който Semantic Kernel се свързва, за да получи реални данни от Airbnb.


## Стартиране на агента с OpenBnB MCP сървър

Сега ще стартираме AI агента, който се свързва с OpenBnB MCP сървъра, за да търси реални Airbnb настанявания в Стокхолм за 2 възрастни и 1 дете. Можете свободно да промените списъка `user_inputs`, за да модифицирате критериите за търсене.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Резюме  
Поздравления! Успешно създадохте AI агент, който се интегрира с реално търсене на настаняване, използвайки Model Context Protocol (MCP):

Използвани технологии:  
Semantic Kernel - За изграждане на интелигентни агенти с Azure OpenAI  
Azure AI Foundry - За възможности на LLM и завършване на чатове  
MCP (Model Context Protocol) - За стандартизирана интеграция на инструменти  
OpenBnB MCP Server - За реална функционалност за търсене в Airbnb  
Node.js/NPX - За стартиране на външния MCP сървър  

Какво научихте:  
MCP интеграция: Свързване на Semantic Kernel агенти с външни MCP сървъри  
Достъп до данни в реално време: Търсене на реални имоти в Airbnb чрез живи API  
Комуникация по протокол: Използване на stdio комуникация между агент и MCP сървър  
Откриване на функции: Автоматично откриване на налични функции от MCP сървъри  
Потокови отговори: Улавяне и записване на извиквания на функции в реално време  
HTML рендиране: Форматиране на отговорите на агента със стилизирани таблици и интерактивни дисплеи  

Следващи стъпки:  
Интегрирайте допълнителни MCP сървъри (време, полети, ресторанти)  
Създайте система с множество агенти, комбинираща MCP и A2A протоколи  
Създайте персонализирани MCP сървъри за ваши собствени източници на данни  
Реализирайте постоянна памет за разговори между сесиите  
Разположете агента в Azure Functions с оркестрация на MCP сървъри  
Добавете потребителска автентикация и възможности за резервации  

Основни предимства на MCP архитектурата:  
Стандартизация: Универсален протокол за свързване на AI агенти с външни инструменти  
Данни в реално време: Достъп до живи, актуални данни от различни услуги  
Разширяемост: Лесна интеграция на нови източници на данни и инструменти  
Съвместимост: Работи с различни AI рамки и платформи за агенти  
Разделение на отговорностите: Ясно разграничение между AI логика и достъп до външни данни  



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
